In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [2]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [3]:
fea_number=4
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A3_018.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A3_019', 'Input_A1_018', 'Input_A2_019', 'Input_A5_019'],
      dtype=object)

## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,4)

train_rmse: 0.00872477114064293
test_rmse: 0.008856737530801326
test_r2: 0.5399154298217665


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,5e-06)

test_rmse_ave: 0.008961732550173195
[0.009007021122923667, 0.00847827463379755, 0.008882503469321276, 0.009459867700218932, 0.00959738986945189, 0.009370382468031763, 0.008393444503329482, 0.009263346279254675, 0.007919152454530228, 0.009245943000872505]


test_r2_ave: 0.5276465264309236
[0.5409520669225529, 0.6093309233020796, 0.5407854401117311, 0.4795289636357467, 0.4758125779702448, 0.431399393593233, 0.5653032341735655, 0.5172070068521097, 0.6133430758579169, 0.5028025818900566]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.035],0.01)

test_rmse_ave: 0.00904063613462884
[0.00901096281088072, 0.008700562033067225, 0.008974751757352439, 0.00955332661020307, 0.009691015613149412, 0.009329832348639322, 0.00850764185169428, 0.009372831123600349, 0.008034117194118081, 0.00923132000358353]


test_r2_ave: 0.519689485578201
[0.540550198289544, 0.5885768822944687, 0.5311976636746388, 0.46919415976264345, 0.4655354461060133, 0.43630995804768036, 0.5533941983113413, 0.5057271624488764, 0.6020351552682077, 0.5043740315785965]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,5,model)

train_rmse: 0.008794517024981087
test_rmse: 0.008955251354206354
test_r2: 0.5294414950675286


## XGB

In [8]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 90, 
              max_depth = 3, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.5,
              reg_alpha =  0.05,
              reg_lambda = 0.1)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.009300400944874419
[0.009087498586840425, 0.008723248964934743, 0.00940858255489126, 0.01026415927878608, 0.009745186994810529, 0.00939392084996118, 0.008872040747426565, 0.01000530800665835, 0.008055483156951258, 0.00944858030748379]


test_r2_ave: 0.491103947356833
[0.5327122603709282, 0.5864284929307498, 0.48477933041493415, 0.38726426863048835, 0.45954358619009406, 0.4285391584859469, 0.514316870734511, 0.43676960229866235, 0.5999156421465206, 0.4807702613654944]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 6,
            min_samples_split = 4,
            max_leaf_nodes = 7,
            min_samples_leaf =9,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.009366969856936084
[0.009272163264035575, 0.008726086437317532, 0.009301632127886646, 0.010225171544754072, 0.010111592235196634, 0.009248994583959785, 0.0092456694647097, 0.00987567232585224, 0.008028712088095762, 0.0096340044975529]


test_r2_ave: 0.48387783912984456
[0.5135280387408299, 0.5861593985622973, 0.49642611844168727, 0.39191030077859057, 0.41813877534716504, 0.4460357565614621, 0.47254830905518685, 0.4512702535938532, 0.6025704520655011, 0.4601909881518721]


In [10]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=5e-06)
elastic_mod=ElasticNet(alpha=[0.035], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 90, 
              max_depth = 3, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.5,
              reg_alpha =  0.05,
              reg_lambda = 0.1)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 6,
            min_samples_split = 4,
            max_leaf_nodes = 7,
            min_samples_leaf =9,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])

# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.009026729630081618
[0.009023207759617822, 0.008456974207416606, 0.009024624615492666, 0.00964393351680469, 0.009615182565159421, 0.009268320680877688, 0.008606508839474994, 0.009457984732714359, 0.007875030655599425, 0.009295528727658504]


test_r2_ave: 0.5207982389933002
[0.5393006621596921, 0.6112914553898228, 0.5259729006918121, 0.4590777364353176, 0.47386718388744087, 0.44371828263369373, 0.5429539070019966, 0.49670527600156966, 0.6176396149150625, 0.4974553708165951]


In [11]:

stack_mod = StackingRegressor(regressors=[svr_mod,elastic_mod,lr, lasso_mod], 
                           meta_regressor=vote_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.008975870380124576
[0.008980723435193114, 0.008422553244467583, 0.008908837805478775, 0.009606110329385667, 0.009573131250644181, 0.009377254987320989, 0.008444156310406526, 0.009337583166640217, 0.007888537919274935, 0.009219815352433765]


test_r2_ave: 0.525988078583147
[0.5436287068339778, 0.61444920271096, 0.5380584978459106, 0.463312374421169, 0.47845912923003975, 0.43056503028571513, 0.5600346338025043, 0.5094377485529826, 0.6163268399005761, 0.5056086222476349]


In [12]:
ave(skew_data[title],y,0.5,stack_mod,0.3,vote_mod,0.2,svr_mod)

test_rmse_ave: 0.008964706593442149
[0.009001469858938365, 0.008341985307651267, 0.008960296243255757, 0.0095318268305453, 0.009529494506012301, 0.009301232389248498, 0.00851788075668307, 0.0093692801088044, 0.00783280075894869, 0.009260799174333855]


test_r2_ave: 0.5271911729498829
[0.5415177391759325, 0.6217900792754745, 0.5327066349153713, 0.47158063045975707, 0.48320292226093153, 0.4397605672363002, 0.5523185756900898, 0.5061016144478739, 0.6217294390577413, 0.5012035269793575]
